<a href="https://colab.research.google.com/github/vathsal14/first-demo/blob/main/pre_processing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install mne


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.6 MB/s eta 0:00:00


In [3]:
import mne
import pandas as pd
import numpy as np
eeg_data = pd.read_csv("data.csv")

In [4]:
eeg_data_cleaned = eeg_data.drop(columns=["Unnamed: 0"])


X = eeg_data_cleaned.drop(columns=["y"]).values
y = eeg_data_cleaned["y"].values


n_channels = X.shape[1]
sfreq = 250
ch_names = [f"Ch{i}" for i in range(1, n_channels + 1)]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types="eeg")


raw = mne.io.RawArray(X.T, info)

Creating RawArray with float64 data, n_channels=178, n_times=3156
    Range : 0 ... 3155 =      0.000 ...    12.620 secs
Ready.


In [5]:
raw.filter(l_freq=1.0, h_freq=50.0)  # Bandpass filter
raw.notch_filter(freqs=50.0)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)



[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.3s


<RawArray | 178 x 3156 (12.6 s), ~4.5 MB, data loaded>

In [6]:

raw.set_eeg_reference("average")


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


<RawArray | 178 x 3156 (12.6 s), ~4.5 MB, data loaded>

In [9]:
X = np.nan_to_num(X)
print(np.isnan(raw.get_data()).any())
print(np.isinf(raw.get_data()).any())
data = raw.get_data()
data[np.isnan(data)] = 0
raw._data = data
print(np.isnan(raw.get_data()).any())
print(np.isinf(raw.get_data()).any())

True
False
False
False


In [11]:
montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage, on_missing="ignore")

print(raw.info['dig'])

raw.filter(1., 40.)
raw.apply_function(lambda x: np.nan_to_num(x), channel_wise=True)

ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter="auto")
ica.fit(raw)
ica.plot_components()

ica.exclude = [0, 1]
raw = ica.apply(raw)

[<DigPoint |        LPA : (-82.5, -0.0, 0.0) mm     : head frame>, <DigPoint |     Nasion : (0.0, 114.0, 0.0) mm      : head frame>, <DigPoint |        RPA : (82.5, 0.0, -0.0) mm      : head frame>]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s


Fitting ICA to data using 178 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.3s
/usr/local/lib/python3.10/dist-packages/mne/preprocessing/ica.py:885: RuntimeWarning: invalid value encountered in divide
  data /= self.pre_whitener_


ValueError: array must not contain infs or NaNs